# Running Stats Analysis

We ran this section to get the statistical signficance numbers. We provide the already extraced csv files which can be used in the summary section.  

In [80]:
import argparse
from tqdm import tqdm

from train import init_model
import torch
import torchvision.transforms as transforms
from torch import nn
from torchvision.models import resnet50, vgg16
from PIL import Image
import scipy.stats as stats
import numpy as np
from argparser import get_args_parser
import utils
from data import get_data_loaders
from scipy.stats import wilcoxon, mannwhitneyu, kruskal
from sklearn.utils import resample

import itertools
import os

Manually change the dataset path. 

In [95]:
parser = argparse.ArgumentParser()
parser.add_argument('arch')
args = get_args_parser().parse_known_args()[0]
data_name = 'BUSI'
# args.data_path = f'dataset/{data_name}'
args.data_path = f'dataset/BUSI/'
num_classes = 3
args.data_path

'dataset/BUSI/'

In [96]:
models = ["resnet50", "vgg16", "vit-ti16", "vit-s16", "vit-s32", "vit-b16", "vit-b32", "vim-s", "vssm-ti", "vssm-s", "vssm-b"]
model_pairs = list(itertools.permutations(models, 2))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Set seeds for reproducibility
utils.set_seed(args.seed)

results = {}
for model_pair in tqdm(model_pairs):
    model1_preds = []
    model2_preds = []
    for fold_index in range(args.k_folds):
        train_loader, val_loader, test_loader = get_data_loaders(args, seed=fold_index + args.seed)
 
        args.arch = model_pair[0]
        model1 = init_model(args, device, num_classes)
        state_dict = torch.load(f'checkpoints/results_{data_name}/{args.arch}/fold_{fold_index}/best_checkpoint.pth')['model']
        model1.load_state_dict(state_dict)

        args.arch = model_pair[1]
        model2 = init_model(args, device, num_classes)
        state_dict = torch.load(f'checkpoints/results_{data_name}/{args.arch}/fold_{fold_index}/best_checkpoint.pth')['model']
        model2.load_state_dict(state_dict)

        model1.eval()
        model2.eval()
        fold_preds1 = []
        fold_preds2 = []
        for data, label in test_loader:
            data = data.to(device)
            label = label.to(device)
            with torch.no_grad():
                pred1 = model1(data)
                pred2 = model2(data)
            pred1 = (torch.argmax(pred1, axis=1) == label).to(torch.int32)
            pred2 = (torch.argmax(pred2, axis=1) == label).to(torch.int32)
            fold_preds1.append(pred1.cpu().numpy())
            fold_preds2.append(pred2.cpu().numpy())
        model1_preds.append(np.concatenate(fold_preds1))
        model2_preds.append(np.concatenate(fold_preds2))
    model1_preds = np.concatenate(model1_preds)
    model2_preds = np.concatenate(model2_preds)
    diff = model1_preds-model2_preds
    t_statistic, p_value_t = stats.ttest_1samp(diff, 0)
    stat_wilcoxon, p_value_wilcoxon = wilcoxon(model1_preds, model2_preds)
    stat_mannwhitney, p_value_mannwhitney = mannwhitneyu(model1_preds, model2_preds)
    stat_kruskal, p_value_kruskal = kruskal(model1_preds, model2_preds)
    
    # Bootstrap sampling for mean difference confidence interval
    mean_diffs = []
    for _ in range(1000):  # Number of bootstrap samples
        sample1 = resample(model1_preds)
        sample2 = resample(model2_preds)
        mean_diffs.append(np.mean(sample1 - sample2))
    ci_lower = np.percentile(mean_diffs, 2.5)
    ci_upper = np.percentile(mean_diffs, 97.5)
    
    print(f"Testing for {model_pair}")

    alpha = 0.05  # commonly used significance level
    # T-test results
    if p_value_t < alpha:
        print(f"  T-test: T={t_statistic:.2f}, p={p_value_t:.3f}. Result is statistically significant.")
    else:
        print(f"  T-test: T={t_statistic:.2f}, p={p_value_t:.3f}. Result is not statistically significant.")

    # Wilcoxon test results
    if p_value_wilcoxon < alpha:
        print(f"  Wilcoxon Signed-Rank Test: Statistic={stat_wilcoxon}, p={p_value_wilcoxon:.3f}. Result is statistically significant.")
    else:
        print(f"  Wilcoxon Signed-Rank Test: Statistic={stat_wilcoxon}, p={p_value_wilcoxon:.3f}. Result is not statistically significant.")

    # Mann-Whitney U test results
    if p_value_mannwhitney < alpha:
        print(f"  Mann-Whitney U Test: U={stat_mannwhitney}, p={p_value_mannwhitney:.3f}. Result is statistically significant.")
    else:
        print(f"  Mann-Whitney U Test: U={stat_mannwhitney}, p={p_value_mannwhitney:.3f}. Result is not statistically significant.")

    # Kruskal-Wallis test results
    if p_value_kruskal < alpha:
        print(f"  Kruskal-Wallis Test: H={stat_kruskal}, p={p_value_kruskal:.3f}. Result is statistically significant.")
    else:
        print(f"  Kruskal-Wallis Test: H={stat_kruskal}, p={p_value_kruskal:.3f}. Result is not statistically significant.")

    # Bootstrap confidence interval
    print(f"  95% CI for mean difference: {ci_lower:.2f} to {ci_upper:.2f}")
    
    results[model_pair] = {
        'T-test': (t_statistic, p_value_t),
        'Wilcoxon': (stat_wilcoxon, p_value_wilcoxon),
        'Mann-Whitney': (stat_mannwhitney, p_value_mannwhitney),
        'Kruskal-Wallis': (stat_kruskal, p_value_kruskal),
        'Bootstrap CI': (ci_lower, ci_upper)
    }

  1%|██▏                                                                                                                                                                                                                                           | 1/110 [00:24<43:54, 24.17s/it]

Testing for ('resnet50', 'vgg16')
  T-test: T=0.12, p=0.901. Result is not statistically significant.
  Wilcoxon Signed-Rank Test: Statistic=1056.0, p=0.901. Result is not statistically significant.
  Mann-Whitney U Test: U=171405.0, p=0.934. Result is not statistically significant.
  Kruskal-Wallis Test: H=0.006910249514144428, p=0.934. Result is not statistically significant.
  95% CI for mean difference: -0.04 to 0.04


  2%|████▎                                                                                                                                                                                                                                         | 2/110 [00:32<26:51, 14.92s/it]

Testing for ('resnet50', 'vit-ti16')
  T-test: T=-0.24, p=0.811. Result is not statistically significant.
  Wilcoxon Signed-Rank Test: Statistic=1207.0, p=0.811. Result is not statistically significant.
  Mann-Whitney U Test: U=170527.5, p=0.867. Result is not statistically significant.
  Kruskal-Wallis Test: H=0.02805644890333662, p=0.867. Result is not statistically significant.
  95% CI for mean difference: -0.04 to 0.03


  3%|██████▍                                                                                                                                                                                                                                       | 3/110 [00:42<22:15, 12.48s/it]

...

Testing for ('vssm-b', 'vim-s')
  T-test: T=1.02, p=0.308. Result is not statistically significant.
  Wilcoxon Signed-Rank Test: Statistic=480.0, p=0.307. Result is not statistically significant.
  Mann-Whitney U Test: U=173160.0, p=0.522. Result is not statistically significant.
  Kruskal-Wallis Test: H=0.40995526928007964, p=0.522. Result is not statistically significant.
  95% CI for mean difference: -0.02 to 0.05


 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊  | 109/110 [31:58<00:23, 23.12s/it]

Testing for ('vssm-b', 'vssm-ti')
  T-test: T=0.45, p=0.655. Result is not statistically significant.
  Wilcoxon Signed-Rank Test: Statistic=483.0, p=0.655. Result is not statistically significant.
  Mann-Whitney U Test: U=171990.0, p=0.781. Result is not statistically significant.
  Kruskal-Wallis Test: H=0.07729834177100475, p=0.781. Result is not statistically significant.
  95% CI for mean difference: -0.03 to 0.04


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 110/110 [32:26<00:00, 17.69s/it]

Testing for ('vssm-b', 'vssm-s')
  T-test: T=1.57, p=0.116. Result is not statistically significant.
  Wilcoxon Signed-Rank Test: Statistic=475.0, p=0.116. Result is not statistically significant.
  Mann-Whitney U Test: U=174330.0, p=0.321. Result is not statistically significant.
  Kruskal-Wallis Test: H=0.987021052412665, p=0.320. Result is not statistically significant.
  95% CI for mean difference: -0.02 to 0.06


In [97]:
import csv
test_names = list(next(iter(results.values())).keys())
final_names = []

for name in test_names:
    if name != 'Bootstrap CI':
        final_names.extend([name+'_stats', name+'_p-value'])
    else:
        final_names.extend([name+'_lower', name+'_upper'])
        
test_names = next(iter(results.values())).keys()
# Open a new CSV file to write the results
with open('results_BUSI.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the header
    writer.writerow(['Model 1', 'Model 2']+final_names)
    
    # Write the data rows
    for (model1, model2), tests in results.items():
        test_values = []
        for test_name in tests:
            test_values.extend(tests[test_name])
        writer.writerow([model1, model2]+test_values)

In [130]:
model_pair = ('resnet50', 'vit-s16')
model1_preds = []
model2_preds = []
args.data_path = f'dataset/Combined/'
for fold_index in range(args.k_folds):
    train_loader, val_loader, test_loader = get_data_loaders(args, seed=fold_index + args.seed)

    args.arch = model_pair[0]
    model1 = init_model(args, device, 3)
    state_dict = torch.load(f'checkpoints/results_Combined/{args.arch}/fold_{fold_index}/best_checkpoint.pth')['model']
    model1.load_state_dict(state_dict)

    args.arch = model_pair[1]
    model2 = init_model(args, device, 3)
    state_dict = torch.load(f'checkpoints/results_Combined/{args.arch}/fold_{fold_index}/best_checkpoint.pth')['model']
    model2.load_state_dict(state_dict)

    model1.eval()
    model2.eval()
    fold_preds1 = []
    fold_preds2 = []
    for data, label in test_loader:
        data = data.to(device)
        label = label.to(device)
        with torch.no_grad():
            pred1 = model1(data)
            pred2 = model2(data)
        pred1 = (torch.argmax(pred1, axis=1) == label).to(torch.int32)
        pred2 = (torch.argmax(pred2, axis=1) == label).to(torch.int32)
        fold_preds1.append(pred1.cpu().numpy())
        fold_preds2.append(pred2.cpu().numpy())
    model1_preds.append(np.concatenate(fold_preds1))
    model2_preds.append(np.concatenate(fold_preds2))
# model1_preds = np.concatenate(model1_preds)
# model2_preds = np.concatenate(model2_preds)

# Summarize

You can use the following code to summarize the  provided csv files for the statistical signficance analysis.

In [1]:
import pandas as pd

In [2]:
p_val_threshold = 0.05
df = pd.read_csv('results_Combined.csv')
model_scores = {i:0 for i in list(df['Model 1'].unique())}
for i in range(df.shape[0]):
    test_stat = df.iloc[i]
    model_name = test_stat['Model 1']
    other_model_name = test_stat['Model 2']
    p_value = test_stat['T-test_p-value']
    t_test = test_stat['T-test_stats']
    if p_value<p_val_threshold:
        if t_test>0:
            model_scores[model_name] += 1  
            print(f"Model (p-value:{p_value:0.3f})-(t_test:{t_test:0.3f})    {model_name}  \t beats '{other_model_name}'")
        else:
            model_scores[model_name] -= 1     
model_scores

Model (p-value:0.044)-(t_test:2.018)    vit-b16  	 beats 'vgg16'
Model (p-value:0.013)-(t_test:2.500)    vit-b16  	 beats 'vit-ti16'
Model (p-value:0.004)-(t_test:2.902)    vssm-ti  	 beats 'vgg16'
Model (p-value:0.003)-(t_test:3.002)    vssm-ti  	 beats 'vit-ti16'
Model (p-value:0.014)-(t_test:2.469)    vssm-ti  	 beats 'vit-s32'
Model (p-value:0.022)-(t_test:2.301)    vssm-ti  	 beats 'vit-b32'
Model (p-value:0.037)-(t_test:2.090)    vssm-b  	 beats 'vgg16'
Model (p-value:0.015)-(t_test:2.438)    vssm-b  	 beats 'vit-ti16'


{'resnet50': 0,
 'vgg16': -3,
 'vit-ti16': -3,
 'vit-s16': 0,
 'vit-s32': -1,
 'vit-b16': 2,
 'vit-b32': -1,
 'vim-s': 0,
 'vssm-ti': 4,
 'vssm-s': 0,
 'vssm-b': 2}

In [4]:
p_val_threshold = 0.05
df = pd.read_csv('results_BUSI.csv')
model_scores = {i:0 for i in list(df['Model 1'].unique())}
for i in range(df.shape[0]):
    test_stat = df.iloc[i]
    model_name = test_stat['Model 1']
    other_model_name = test_stat['Model 2']
    p_value = test_stat['T-test_p-value']
    t_test = test_stat['T-test_stats']
    if p_value<p_val_threshold:
        if t_test>0:
            model_scores[model_name] += 1  
            print(f"Model (p-value:{p_value:0.3f})-(t_test:{t_test:0.3f})    {model_name}  \t beats '{other_model_name}'")
        else:
            model_scores[model_name] -= 1  
model_scores

Model (p-value:0.036)-(t_test:2.099)    vit-b16  	 beats 'vit-s32'
Model (p-value:0.011)-(t_test:2.535)    vim-s  	 beats 'vit-s32'
Model (p-value:0.032)-(t_test:2.148)    vssm-ti  	 beats 'resnet50'
Model (p-value:0.024)-(t_test:2.258)    vssm-ti  	 beats 'vgg16'
Model (p-value:0.002)-(t_test:3.086)    vssm-ti  	 beats 'vit-s32'
Model (p-value:0.027)-(t_test:2.223)    vssm-s  	 beats 'vit-s32'
Model (p-value:0.015)-(t_test:2.436)    vssm-b  	 beats 'resnet50'
Model (p-value:0.006)-(t_test:2.749)    vssm-b  	 beats 'vgg16'
Model (p-value:0.018)-(t_test:2.373)    vssm-b  	 beats 'vit-ti16'
Model (p-value:0.022)-(t_test:2.296)    vssm-b  	 beats 'vit-s16'
Model (p-value:0.000)-(t_test:3.819)    vssm-b  	 beats 'vit-s32'
Model (p-value:0.029)-(t_test:2.190)    vssm-b  	 beats 'vit-b32'


{'resnet50': -2,
 'vgg16': -2,
 'vit-ti16': -1,
 'vit-s16': -1,
 'vit-s32': -5,
 'vit-b16': 1,
 'vit-b32': -1,
 'vim-s': 1,
 'vssm-ti': 3,
 'vssm-s': 1,
 'vssm-b': 6}

In [5]:
p_val_threshold = 0.05
df = pd.read_csv('results_B.csv')
model_scores = {i:0 for i in list(df['Model 1'].unique())}
for i in range(df.shape[0]):
    test_stat = df.iloc[i]
    model_name = test_stat['Model 1']
    other_model_name = test_stat['Model 2']
    p_value = test_stat['T-test_p-value']
    t_test = test_stat['T-test_stats']
    if p_value<p_val_threshold:
        if t_test>0:
            model_scores[model_name] += 1  
            print(f"Model (p-value:{p_value:0.3f})-(t_test:{t_test:0.3f})    {model_name}  \t beats '{other_model_name}'")
        else:
            model_scores[model_name] -= 1  
model_scores

Model (p-value:0.034)-(t_test:2.142)    vit-s16  	 beats 'vit-b16'
Model (p-value:0.032)-(t_test:2.171)    vim-s  	 beats 'vit-ti16'
Model (p-value:0.019)-(t_test:2.368)    vim-s  	 beats 'vit-b16'
Model (p-value:0.011)-(t_test:2.582)    vssm-ti  	 beats 'resnet50'
Model (p-value:0.028)-(t_test:2.218)    vssm-ti  	 beats 'vgg16'
Model (p-value:0.004)-(t_test:2.915)    vssm-ti  	 beats 'vit-ti16'
Model (p-value:0.012)-(t_test:2.557)    vssm-ti  	 beats 'vit-s32'
Model (p-value:0.003)-(t_test:3.087)    vssm-ti  	 beats 'vit-b16'
Model (p-value:0.007)-(t_test:2.756)    vssm-ti  	 beats 'vit-b32'
Model (p-value:0.034)-(t_test:2.142)    vssm-ti  	 beats 'vssm-b'


{'resnet50': -1,
 'vgg16': -1,
 'vit-ti16': -2,
 'vit-s16': 1,
 'vit-s32': -1,
 'vit-b16': -3,
 'vit-b32': -1,
 'vim-s': 2,
 'vssm-ti': 7,
 'vssm-s': 0,
 'vssm-b': -1}